In [1]:
import numpy as np
import pandas as pd
import functools
import itertools
import operator
import gurobipy as gp
import random
import math
from docplex.cp.model import CpoModel

In [2]:
containers = pd.read_excel (r'C:\Users\User\Desktop\Containers\Dataset2\Контейнеры (пункт назначения).xlsx');
platforms = pd.read_excel (r'C:\Users\User\Desktop\Containers\Dataset2\Платформы (пункт назначения).xlsx');

In [3]:
containers['Масса'] = containers['МассаГруза'] + containers['МассаТары']

In [4]:
containers['МассаГруза'] = containers['МассаГруза'].str.replace(" ","")
containers['МассаГруза'] = pd.to_numeric(containers['МассаГруза'])

In [5]:
containers['МассаТары'] = containers['МассаТары'].str.replace(" ","")
containers['МассаТары'] = pd.to_numeric(containers['МассаТары'])

In [6]:
containers['Масса'] = (containers['МассаГруза'] + containers['МассаТары'])/1000

In [7]:
containersIndex = containers.set_index('Контейнер')

In [8]:
containersDict = containersIndex.T.to_dict('list')

In [9]:
platforms['Грузоподъемность'] = platforms['Грузоподъемность'].str.replace(",",".")
platforms['Грузоподъемность'] = pd.to_numeric(platforms['Грузоподъемность'])

In [10]:
platformsDict = platforms.set_index('Вагон').T.to_dict('list')

In [11]:
cardinalityRange = [1,4]

In [12]:
def powerset(iterable,cardinalityRange):
    return list(itertools.chain.from_iterable(itertools.combinations(iterable, r) for r in range(cardinalityRange[0],cardinalityRange[1])))

In [13]:
allSubsets = powerset(list(containersDict.keys()),cardinalityRange)

In [14]:
containersGroups = containersIndex.groupby(['Партия'])

In [15]:
def functools_reduce_iconcat(a):
    return functools.reduce(operator.iconcat, a, [])

In [16]:
subsetsByBatches = functools_reduce_iconcat([powerset(list(c),cardinalityRange) for c in containersGroups.groups.values()])

In [17]:
maxQ = max(platforms['Грузоподъемность'])

In [18]:
subsetsByBatchesFilter1 = list(filter(lambda x: sum([containersDict[c][7] for c in x]) <= maxQ, subsetsByBatches))

In [19]:
variants = functools_reduce_iconcat([list(itertools.permutations(c)) for c in subsetsByBatchesFilter1])

In [20]:
numberOfPlatforms = len(platformsDict)
numberOfContainers = len(containersDict)
share = 0.25

In [21]:
subsetsDict = dict((key, random.sample(variants,random.randint(1,math.floor(share*len(subsetsByBatchesFilter1))))) for key in range(0,numberOfPlatforms))

In [22]:
step1 = functools_reduce_iconcat([np.asarray(tup) for tup in functools_reduce_iconcat(subsetsDict.values())])

In [23]:
containersNames = list(set(step1))

In [24]:
from docplex.mp.model import Model

In [53]:
model = Model("Containers")

In [54]:
idx = [(key, num) for key in subsetsDict.keys() for num in range(0,len(subsetsDict[key]))]

In [55]:
idsBatches = sorted(containers['Партия'].unique())

In [56]:
varsx = model.binary_var_dict(idx, name = 'X')

In [57]:
varsy2 = model.binary_var_dict(idsBatches, name = 'Y2')

In [58]:
myCost1 = model.sum(-varsx[i] for i in idx)
model.add_kpi(myCost1, 'Platforms')

DecisionKPI(name=Platforms,expr=-X_0_0-X_0_1-X_0_2-X_0_3-X_0_4-X_0_5-X_0_6-X_0_7-X_0_8-X_0_9-X_0..)

In [59]:
myCost2 = model.sum(len(subsetsDict[i][j])*varsx[i,j] for (i,j) in idx)
model.add_kpi(myCost2, 'Containers')

DecisionKPI(name=Containers,expr=3X_0_0+3X_0_1+3X_0_2+3X_0_3+3X_0_4+3X_0_5+3X_0_6+3X_0_7+3X_0_8+3..)

In [60]:
for c in containersNames:
    listOfPositions = []
    for key in subsetsDict.keys():
        for i,x in enumerate(subsetsDict[key]):
            if c in x:
                listOfPositions.append((key,i))
    if len(listOfPositions) > 0:
        model.add_constraint(model.sum(varsx[pos] for pos in listOfPositions) <= 1)

In [61]:
 for key in subsetsDict.keys():
         model.add_constraint(model.sum(varsx[key,pos] for pos in range(0,len(subsetsDict[key]))) <= 1)

In [73]:
batchesDict = {id:[] for id in idsBatches}

In [74]:
for key in subsetsDict.keys():
    for i,x in enumerate(subsetsDict[key]):
        batchesInX = [containersDict[c][5] for c in x]
        for b in set(batchesInX):
            batchesDict[b].append((key,i,batchesInX.count(b)))

In [64]:
batchesCounts = containers['Партия'].value_counts()

In [76]:
for b in idsBatches:
    model.add_constraint(model.sum(k*varsx[i,j] for (i,j,k) in batchesDict[b]) - batchesCounts[b]*varsy2[b] == 0 )

In [77]:
model.maximize_static_lex([myCost2, myCost1])

In [78]:
model.solve()

docplex.mp.solution.SolveSolution(obj=59,values={X_0_273:1,X_1_1520:1,X_..

In [79]:
solution = {}
for v in model.iter_binary_vars():
    if v.solution_value == 1:
        name = v.name.split('_')
        if name[0] == 'X':
            solution[list(subsetsDict.keys())[int(name[1])]] =  subsetsDict[int(name[1])][int(name[2])]

In [80]:
solution

{0: ('PONU 0233749', 'MSCU 6376558', 'MRKU 7402432'),
 1: ('MRKU 7209130', 'TCLU 2178392', 'MSKU 7252843'),
 2: ('GLDU 5214557', 'MEDU 2752503', 'MSKU 2334933'),
 3: ('TGHU 0608214', 'MSKU 2442753', 'MRSU 4368888'),
 4: ('TEMU 0116038', 'MRKU 8929639', 'BEAU 5112926'),
 5: ('SUDU 7789029', 'MSKU 2019269', 'DRYU 2439546'),
 6: ('MSKU 2869806', 'SUDU 1680956', 'MSDU 2147639'),
 7: ('MSDU 2038047', 'MSKU 5163217', 'MRKU 5100937'),
 8: ('MSKU 1483552', 'MRKU 6687325', 'MSKU 4425349'),
 9: ('TCKU 1442327', 'TCLU 2410316', 'MRKU 3142527'),
 10: ('MRKU 9627443', 'MRKU 9724720', 'AMFU 3254656'),
 11: ('MRKU 7043841', 'MRKU 9374514', 'CARU 2216790'),
 15: ('MRKU 9033020', 'MSKU 3887851'),
 17: ('MRKU 7062395', 'SEKU 4629612', 'TEMU 1409370'),
 18: ('MRKU 9232361', 'MRKU 6854058', 'TRHU 1726358'),
 19: ('HASU 1177403', 'MSKU 9310569', 'MRKU 7799000'),
 20: ('MSKU 7398416', 'PONU 0250788', 'MSKU 7848874'),
 21: ('HASU 1434986', 'MRKU 9164858', 'TCLU 7486607'),
 22: ('MSKU 7453895',),
 26: ('MSKU 

In [81]:
len(solution)

21

In [87]:
allContainersToDeparture = functools_reduce_iconcat([np.asarray(tup) for tup in solution.values()])

In [88]:
len(allContainersToDeparture)

59

In [91]:
len(set(allContainersToDeparture))

59